In [1]:
from mand.core import Entity, node, getNode
from mand.core import ObjectDb, Timestamp, Context, _tr
from mand.core import ProfileMonitor, SummaryMonitor
from mand.core import find, addFootnote
from mand.demos.trading import makeWorld, bookSomeTrades, PnLExplainReport
from mand.lib.dbsetup import setUpDb

db = ObjectDb()
setUpDb(db)

In [2]:
with db:
    pWorld = makeWorld()
    
pAll, bExt, bExt2 = pWorld.children()
p1 = pAll.children()[0]
p2 = pAll.children()[1]
p4 = pAll.children()[3]

b2 = p2.children()[0]
b4 = p4.children()[0]

makeWorld, TopOfTheHouse is: <Entity:/Global/TradingPortfolio/TopOfTheHouse>
    # books: 100
    # children: 10


In [3]:
ts0, ts1, ts2, ts3, ts4, ts5, eod, ts6 = bookSomeTrades(pWorld)

p1.setChildren(p1.children() + [b2])
ts7 = Timestamp()

# Sanity check of a computation we might want to optimize [Test]

The PnL report is a good test, but at 152K getValues, it's a bit too big to immediately use:

In [4]:
db3 = db.copy()
p = db3._get(pAll.meta.path())

r = PnLExplainReport(valuable=p, ts1=eod, ts2=ts7)

with ProfileMonitor():
    with SummaryMonitor():
        r.run()

# PnL explain for TopOfTheHouse: 6256.00

**Caveat: this report encountered problems. See footnotes at bottom.**

|Activity|PnL|
|-|-|
|Starting balance breaks|0.00
|prior day amends: MarketData|0.00
|prior day amends: Portfolio|0.00
|prior day amends: Trading|-4.00
|activity: MarketData|5240.00
|activity: Portfolio|1020.00
|activity: Trading|0.00
|Ending balance breaks|0.00

## Footnotes:
1. Inadequate cash discounting model used
1. Book appears multiple times
  * /Global/TradingBook/Eq-Inst0

# Compute activity summary (33.37 seconds of wall clock time)

|key|value|
|-|-|
|Context|11
|Db.Get|2176
|GetValue|152158
|GetValue/Calc|2891


### Profile by nodes.
* times are in microseconds
* cumT is total time spent in funtion
* calcT is time spent in function, but not in a child node

|fn|n|cumT|calcT|cumT/call|sys|
|-|-|-|-|-|-|
|PnLExplainReport:data|1|33,346,873|27|33,346,873|GetValue
|PnLExplainReport:data|1|33,346,845|807|33,346,845|GetValue/Calc
|TradingContainer:NPV|11|33,244,371|257|3,022,215|GetValue
|TradingContainer:NPV|11|33,244,113|159|3,022,192|GetValue/Calc
|Portfolio:items|121|32,952,086|2,269|272,331|GetValue
|Portfolio:items|121|32,951,903|1,637|272,329|GetValue/Calc
|Workbook:items|1,104|31,246,024|21,713|28,302|GetValue
|Workbook:items|1,100|31,224,310|17,114|28,385|GetValue/Calc
|PnLExplainReport:cutoffs|1|18,008,327|31|18,008,327|GetValue
|PnLExplainReport:cutoffs|1|18,008,296|265|18,008,296|GetValue/Calc
|Root:Clocks|548|18,007,879|3,220,824|32,861|Context
|TradeOpenEvent|1,014|7,378,299|7,378,299|7,276|Db.Get
|TradeOpenEvent:ticket|22,286|6,420,828|221,852|288|GetValue
|TradingTicket|1,014|6,198,976|6,198,976|6,113|Db.Get
|Root:Activity Portfolio|263|1,985,954|1,836,719|7,551|Context
|Root:End|278|1,777,172|1,621,580|6,392|Context
|Root:Amend Trading|255|1,776,880|1,640,272|6,968|Context
|Root:Amend MarketData|255|1,758,508|1,620,258|6,896|Context
|Root:Activity Trading|263|1,722,955|1,568,432|6,551|Context
|Root:Start breaks|255|1,647,405|1,515,661|6,460|Context
|Portfolio:children|242|1,628,362|3,466|6,728|GetValue
|Portfolio:children|121|1,624,895|1,745|13,428|GetValue/Calc
|Root:Activity MarketData|255|1,607,132|1,463,659|6,302|Context
|Root:Amend Portfolio|255|1,544,911|1,407,363|6,058|Context
|Root:Start|270|1,510,550|1,378,926|5,594|Context
|PortfolioUpdateEvent:children|121|664,531|5,105|5,491|GetValue
|TradingBook|102|611,799|611,799|5,998|Db.Get
|Equity:NPV|15|287,924|357|19,194|GetValue
|Equity:NPV|15|287,566|222|19,171|GetValue/Calc
|MarketInterface:spot|15|272,759|290|18,183|GetValue
|MarketInterface:spot|15|272,469|207|18,164|GetValue/Calc
|ExternalRefData:state|15|257,442|247|17,162|GetValue
|ExternalRefData:state|15|257,194|207|17,146|GetValue/Calc
|RefData:state|15|256,311|262|17,087|GetValue
|RefData:state|15|256,048|220|17,069|GetValue/Calc
|_WorkItemEvent:book2|22,286|212,961|200,808|9|GetValue
|_WorkItemEvent:book1|22,286|206,007|206,007|9|GetValue
|TradeOpenEvent:quantity|22,286|184,289|184,289|8|GetValue
|Clock:cutoffs|2,480|179,704|17,982|72|GetValue
|Clock:cutoffs|20|161,860|268|8,093|GetValue/Calc
|Clock:parent|20|160,107|345|8,005|GetValue
|Clock:parent|20|159,762|290|7,988|GetValue/Calc
|_WorkItemEvent:item|11,143|151,168|138,293|13|GetValue
|TradeOpenEvent:action|11,143|132,937|132,937|11|GetValue
|TradeOpenEvent:premium|11,143|131,807|125,725|11|GetValue
|TradeOpenEvent:unitPrice|11,143|111,671|111,671|10|GetValue
|TradingBook:clock|2,200|106,044|41,314|48|GetValue
|Event:amends|11,340|105,500|105,500|9|GetValue
|PortfolioUpdateEvent|12|85,692|85,692|7,141|Db.Get
|TradingBook:clock|1,100|64,730|22,095|58|GetValue/Calc
|RefDataUpdateEvent|9|61,884|61,884|6,876|Db.Get
|TradingPortfolio|10|59,779|59,779|5,977|Db.Get
|Clock|5|31,201|31,201|6,240|Db.Get
|Portfolio:books|231|17,237|3,388|74|GetValue
|Portfolio:clock|242|16,340|4,413|67|GetValue
|Portfolio:books|121|14,604|1,727|120|GetValue/Calc
|MarketInterface:source|15|13,722|253|914|GetValue
|ClockEvent|2|13,715|13,715|6,857|Db.Get
|Equity:refdata|15|13,516|289|901|GetValue
|MarketInterface:source|15|13,468|239|897|GetValue/Calc
|ClockEvent:parent|8|13,460|153|1,682|GetValue
|Equity:refdata|15|13,226|209|881|GetValue/Calc
|Equity|2|12,875|12,875|6,437|Db.Get
|Portfolio:clock|121|11,927|2,431|98|GetValue/Calc
|MarketInterface|2|11,910|11,910|5,955|Db.Get
|MarketDataSource|2|11,624|11,624|5,812|Db.Get
|MarketDataSource:clock|30|7,276|495|242|GetValue
|MarketDataSource:clock|15|6,780|293|452|GetValue/Calc
|RootClock|1|6,203|6,203|6,203|Db.Get
|ForwardCashflow|1|6,082|6,082|6,082|Db.Get
|Entity:clock|40|1,604|671|40|GetValue
|Entity:clock|20|932|387|46|GetValue/Calc
|MarketInterface:sourceName|15|722|291|48|GetValue
|ForwardCashflow:NPV|11|658|225|59|GetValue
|RefDataUpdateEvent:data|58|535|535|9|GetValue
|RootClock:cutoffs|53|487|373|9|GetValue
|ForwardCashflow:NPV|11|432|161|39|GetValue/Calc
|MarketInterface:sourceName|15|430|214|28|GetValue/Calc
|Equity:assetName|15|201|201|13|GetValue
|RootClock:cutoffs|1|113|55|113|GetValue/Calc
|RootClock:cosmicAll|1|32|18|32|GetValue
|CosmicAll:dbState|1|25|16|25|GetValue
|PnLExplainReport:ts1|2|22|22|11|GetValue
|PnLExplainReport:valuable|2|17|17|8|GetValue
|PnLExplainReport:ts2|2|15|15|7|GetValue
|RootClock:cosmicAll|1|14|14|14|GetValue/Calc
|CosmicAll:dbState|1|9|9|9|GetValue/Calc

# Caching/Reusing results

Until now, we have just checked to see if the current context contains a value for a node we are asking for, and if so, reuse that.

A better approach to caching bound *fn* on *object* in *context0* is:
    
1. Has fn been tweaked in context0? If so, return that
2. Is fn sufficiently trivial that we can avoid managing it as a node?
  * If trivial, just treat it as a pure python function and call it
  * Meta-data (inputs, outputs, etc) will be given to its caller and callees as appropriate
3. Ask object for context1
  * *context1* is a simplified version of *context0*
  * The default case is just to return *context1*
  * IRL, this could actually be a list of contexts or a pattern to match contexts against
4. Is fn cached in *context1*? If so, use that
5. Compute *fn* in *context1*
6. Construct *context2* from the inputs of the computed value
7. If *context2* is a subset of *context1*, cache *fn* in *context1*
8. Something odd happened
  * Footnote the problem as part of computation notes on the node's metadata
  * Maybe cache the node anyway in *context2*
    * Perhaps *object* will return *context2* as a siplification for future computations?
9. Return the value

Notes:
* Parallel compute of nodes not considered yet
* Context simplification (step 3) and input simplification (step 9) are probably intimately related
* The split between calculation and caching is nice:
  * BAs can write business logic
  * Computer scientists can add caching logic where needed
* We can use a profiler type object to gather runtime compute cost infomation
  * The resulting trace can be used as input to drive step 2 and step 3
    

In [5]:
dba = db.copy()
valuable = dba.get(b4.meta.path())

clock = valuable.getObj(_tr.RootClock, 'Main')
with SummaryMonitor():
    with Context({clock.cutoffs: ts7}):
        print valuable.NPV()
        node = getNode(valuable.NPV)
node.printInputGraph()

40.85


# Compute activity summary (0.33 seconds of wall clock time)

|key|value|
|-|-|
|Context|1
|Db.Get|35
|GetValue|185
|GetValue/Calc|24

<Node: /Global/TradingBook/Rates0.(TradingContainer:NPV)() in Root:4478373784 @4581901008 T=False>, nIn=3
   <Node: /Global/Equity/4b90c5a7-2095-47b7-8306-110b555da62f.(Equity:NPV)() in Root:4478373784 @4624176720 T=False>, nIn=2
      <Node: /Global/Equity/4b90c5a7-2095-47b7-8306-110b555da62f.(Equity:refdata)() in Root:4478373784 @4624176784 T=False>, nIn=1
         <Node: /Global/Equity/4b90c5a7-2095-47b7-8306-110b555da62f.(Equity:assetName)() in Root:4478373784 @4624176912 T=False>, nIn=0
      <Node: /Global/MarketInterface/GOOG.(MarketInterface:spot)() in Root:4478373784 @4624177808 T=True>, nIn=2
         <Node: /Global/MarketDataSource/source1.GOOG.(ExternalRefData:state)() in Root:4478373784 @4624178960 T=False>, nIn=1
            <Node: /Global/MarketDataSource/source1.GOOG.(RefData:state)() in Root:4478373784 @4643647696 T=False>, nIn=10
               <Node: /Global/Clock/MarketData.(Clock:cutoffs)() in Root:4478373784 @4605196560 T=True>, nIn=2
                  <Node: /Glo

# Graph Simplification

* Very simple input management simplifier: ignore inputs if they are leaves and not tweakable
* Simple context simplification: just switch to a simple context for certain known methods

In [6]:
from mand.graph import DependencyManager, setDependencyManager
from mand.core import Event

class DM1(DependencyManager):
    
    def __init__(self):
        self.contexts = {}
        self.simpleClockMethods = ('Workbook:items', 'RefData:state', 'Portfolio:children', 'Portfolio:items')
        super(DM1, self).__init__()
        
    def addDep(self, input, output):
        if not input.key.tweakable:
            if not input.inputs:
                return
        output.inputs.add(input)
        input.outputs.add(output)


    def calculated(self, node):
        if not node.isSimplified:
            return
        for input in node.inputs:
            if input not in node.ctx.tweaks:
                addFootnote(text='context simplification failure', info='%s used %s' % (str(node.key), str(input.key)))
                  
    def getCtx(self, clock):
        ts = clock.cutoffs()
        cKey = (clock, ts) # really want value-based comparison of ts, not object equality
        if cKey not in self.contexts:
            self.contexts[cKey] = Context({clock.cutoffs: ts})
        return self.contexts[cKey]
            
    def getNode(self, ctx, key):
        n = ctx._get(key)
        if n:
            return n
        if key.fullName() in self.simpleClockMethods:
            obj = key.object()
            ctx1 = self.getCtx(obj.clock())
            ret = ctx1.get(key)
            ret.isSimplified = True
        else:
            ret = super(DM1, self).getNode(ctx, key)
            ret.isSimplified = False
        return ret
    
setDependencyManager(DM1())

db4 = db.copy()
valuable = db4.get(b4.meta.path())

clock = valuable.getObj(_tr.RootClock, 'Main')
with Context({clock.cutoffs: ts7}):
    print valuable.NPV()
    node = getNode(valuable.NPV)
    node.printInputGraph()

40.85
<Node: /Global/TradingBook/Rates0.(TradingContainer:NPV)() in Root:4478373784 @4699925840 T=False>, nIn=3
   <Node: /Global/Clock/Trading.(Clock:cutoffs)() in Root:4478373784 @4758295568 T=True>, nIn=2
      <Node: /Global/Clock/Trading.(Clock:parent)() in Root:4478373784 @4758294672 T=False>, nIn=1
         <Node: /Global/RootClock/Main.(RootClock:cutoffs)() in Root:4478373784 @4699926224 T=True>, nIn=0
      <Node: /Global/Clock/Workflow.(Clock:cutoffs)() in Root:4478373784 @4740063504 T=True>, nIn=2
         <Node: /Global/Clock/Workflow.(Clock:parent)() in Root:4478373784 @4740063888 T=False>, nIn=1
            <Node: /Global/RootClock/Main.(RootClock:cutoffs)() in Root:4478373784 @4699926224 T=True>, nIn=0
         <Node: /Global/RootClock/Main.(RootClock:cutoffs)() in Root:4478373784 @4699926224 T=True>, nIn=0
   <Node: /Global/Equity/4b90c5a7-2095-47b7-8306-110b555da62f.(Equity:NPV)() in Root:4478373784 @4601141584 T=False>, nIn=1
      <Node: /Global/MarketInterface/GOOG.

In [7]:
db4 = db.copy()
valuable = db4.get(pAll.meta.path())

r = PnLExplainReport(valuable=valuable, ts1=eod, ts2=ts7)
with ProfileMonitor():
    with SummaryMonitor():
        r.run()

# PnL explain for TopOfTheHouse: 6256.00

**Caveat: this report encountered problems. See footnotes at bottom.**

|Activity|PnL|
|-|-|
|Starting balance breaks|0.00
|prior day amends: MarketData|0.00
|prior day amends: Portfolio|0.00
|prior day amends: Trading|0.00
|activity: MarketData|5240.00
|activity: Portfolio|1016.00
|activity: Trading|0.00
|Ending balance breaks|0.00

## Footnotes:
1. context simplification failure
  * <NodeKey:<Entity:/Global/TradingPortfolio/Commod>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/Clock/Trading>.(Clock:cutoffs) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Commod>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Commod0>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Commod>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Commod1>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Commod>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Commod2>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Commod>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Commod3>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Commod>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Commod4>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Commod>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Commod5>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Commod>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Commod6>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Commod>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Commod7>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Commod>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Commod8>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Commod>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Commod9>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Commod>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingPortfolio/Commod>.(Portfolio:books) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Commod>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingPortfolio/Commod>.(Portfolio:children) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Credit>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/Clock/Trading>.(Clock:cutoffs) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Credit>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Credit0>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Credit>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Credit1>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Credit>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Credit2>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Credit>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Credit3>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Credit>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Credit4>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Credit>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Credit5>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Credit>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Credit6>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Credit>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Credit7>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Credit>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Credit8>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Credit>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Credit9>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Credit>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingPortfolio/Credit>.(Portfolio:books) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Credit>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingPortfolio/Credit>.(Portfolio:children) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Delta1>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/Clock/Trading>.(Clock:cutoffs) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Delta1>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Delta10>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Delta1>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Delta11>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Delta1>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Delta12>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Delta1>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Delta13>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Delta1>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Delta14>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Delta1>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Delta15>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Delta1>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Delta16>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Delta1>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Delta17>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Delta1>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Delta18>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Delta1>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Delta19>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Delta1>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingPortfolio/Delta1>.(Portfolio:books) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Delta1>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingPortfolio/Delta1>.(Portfolio:children) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/ETFs>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/Clock/Trading>.(Clock:cutoffs) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/ETFs>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/ETFs0>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/ETFs>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/ETFs1>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/ETFs>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/ETFs2>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/ETFs>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/ETFs3>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/ETFs>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/ETFs4>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/ETFs>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/ETFs5>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/ETFs>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/ETFs6>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/ETFs>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/ETFs7>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/ETFs>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/ETFs8>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/ETFs>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/ETFs9>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/ETFs>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingPortfolio/ETFs>.(Portfolio:books) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/ETFs>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingPortfolio/ETFs>.(Portfolio:children) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Eq-Inst>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/Clock/Trading>.(Clock:cutoffs) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Eq-Inst>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Eq-Inst0>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Eq-Inst>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Eq-Inst1>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Eq-Inst>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Eq-Inst2>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Eq-Inst>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Eq-Inst3>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Eq-Inst>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Eq-Inst4>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Eq-Inst>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Eq-Inst5>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Eq-Inst>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Eq-Inst6>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Eq-Inst>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Eq-Inst7>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Eq-Inst>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Eq-Inst8>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Eq-Inst>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Eq-Inst9>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Eq-Inst>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingPortfolio/Eq-Inst>.(Portfolio:books) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Eq-Inst>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingPortfolio/Eq-Inst>.(Portfolio:children) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Eq-Prop>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/Clock/Trading>.(Clock:cutoffs) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Eq-Prop>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Eq-Inst0>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Eq-Prop>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Eq-Prop0>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Eq-Prop>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Eq-Prop1>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Eq-Prop>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Eq-Prop2>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Eq-Prop>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Eq-Prop3>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Eq-Prop>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Eq-Prop4>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Eq-Prop>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Eq-Prop5>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Eq-Prop>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Eq-Prop6>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Eq-Prop>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Eq-Prop7>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Eq-Prop>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Eq-Prop8>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Eq-Prop>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Eq-Prop9>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Eq-Prop>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingPortfolio/Eq-Prop>.(Portfolio:books) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Eq-Prop>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingPortfolio/Eq-Prop>.(Portfolio:children) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/FX>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/Clock/Trading>.(Clock:cutoffs) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/FX>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/FX0>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/FX>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/FX1>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/FX>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/FX2>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/FX>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/FX3>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/FX>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/FX4>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/FX>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/FX5>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/FX>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/FX6>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/FX>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/FX7>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/FX>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/FX8>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/FX>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/FX9>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/FX>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingPortfolio/FX>.(Portfolio:books) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/FX>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingPortfolio/FX>.(Portfolio:children) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Loans>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/Clock/Trading>.(Clock:cutoffs) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Loans>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Loans0>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Loans>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Loans1>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Loans>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Loans2>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Loans>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Loans3>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Loans>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Loans4>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Loans>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Loans5>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Loans>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Loans6>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Loans>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Loans7>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Loans>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Loans8>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Loans>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Loans9>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Loans>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingPortfolio/Loans>.(Portfolio:books) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Loans>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingPortfolio/Loans>.(Portfolio:children) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Mtge>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/Clock/Trading>.(Clock:cutoffs) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Mtge>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Mtge0>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Mtge>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Mtge1>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Mtge>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Mtge2>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Mtge>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Mtge3>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Mtge>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Mtge4>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Mtge>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Mtge5>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Mtge>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Mtge6>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Mtge>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Mtge7>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Mtge>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Mtge8>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Mtge>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Mtge9>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Mtge>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingPortfolio/Mtge>.(Portfolio:books) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Mtge>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingPortfolio/Mtge>.(Portfolio:children) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Rates>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/Clock/Trading>.(Clock:cutoffs) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Rates>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Rates0>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Rates>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Rates1>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Rates>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Rates2>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Rates>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Rates3>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Rates>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Rates4>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Rates>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Rates5>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Rates>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Rates6>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Rates>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Rates7>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Rates>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Rates8>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Rates>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingBook/Rates9>.(Workbook:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Rates>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingPortfolio/Rates>.(Portfolio:books) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/Rates>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingPortfolio/Rates>.(Portfolio:children) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/TopOfTheHouse>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingPortfolio/Commod>.(Portfolio:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/TopOfTheHouse>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingPortfolio/Credit>.(Portfolio:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/TopOfTheHouse>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingPortfolio/Delta1>.(Portfolio:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/TopOfTheHouse>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingPortfolio/ETFs>.(Portfolio:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/TopOfTheHouse>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingPortfolio/Eq-Inst>.(Portfolio:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/TopOfTheHouse>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingPortfolio/Eq-Prop>.(Portfolio:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/TopOfTheHouse>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingPortfolio/FX>.(Portfolio:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/TopOfTheHouse>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingPortfolio/Loans>.(Portfolio:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/TopOfTheHouse>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingPortfolio/Mtge>.(Portfolio:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/TopOfTheHouse>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingPortfolio/Rates>.(Portfolio:items) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/TopOfTheHouse>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingPortfolio/TopOfTheHouse>.(Portfolio:books) ()>
  * <NodeKey:<Entity:/Global/TradingPortfolio/TopOfTheHouse>.(Portfolio:items) ()> used <NodeKey:<Entity:/Global/TradingPortfolio/TopOfTheHouse>.(Portfolio:children) ()>
1. Inadequate cash discounting model used
1. Book appears multiple times
  * /Global/TradingBook/Eq-Inst0

# Compute activity summary (29.40 seconds of wall clock time)

|key|value|
|-|-|
|Context|19
|Db.Get|2176
|GetValue|28897
|GetValue/Calc|1031


### Profile by nodes.
* times are in microseconds
* cumT is total time spent in funtion
* calcT is time spent in function, but not in a child node

|fn|n|cumT|calcT|cumT/call|sys|
|-|-|-|-|-|-|
|PnLExplainReport:data|1|29,380,333|44|29,380,333|GetValue
|PnLExplainReport:data|1|29,380,289|1,314|29,380,289|GetValue/Calc
|TradingContainer:NPV|11|29,359,666|529|2,669,060|GetValue
|TradingContainer:NPV|11|29,359,136|355|2,669,012|GetValue/Calc
|PnLExplainReport:cutoffs|1|29,033,841|48|29,033,841|GetValue
|PnLExplainReport:cutoffs|1|29,033,792|444|29,033,792|GetValue/Calc
|Root:Clocks|47|29,033,155|87,615|617,726|Context
|Portfolio:items|41|28,845,780|25,220|703,555|GetValue
|Portfolio:items|33|28,843,796|1,059|874,054|GetValue/Calc
|Root:Clocks:4478373784|306|28,841,394|440,539|94,252|Context
|Workbook:items|301|26,883,624|10,760|89,314|GetValue
|Workbook:items|200|26,872,864|6,383|134,364|GetValue/Calc
|Root:Clocks:4478373784:4478373784|400|26,866,481|6,009,269|67,166|Context
|TradeOpenEvent:ticket|4,052|10,564,655|108,469|2,607|GetValue
|TradingTicket|1,014|10,456,186|10,456,186|10,311|Db.Get
|TradeOpenEvent|1,014|9,755,701|9,755,701|9,621|Db.Get
|Portfolio:children|66|1,743,216|2,134|26,412|GetValue
|Portfolio:children|33|1,741,081|1,037|52,760|GetValue/Calc
|PortfolioUpdateEvent:children|33|1,161,509|4,865|35,197|GetValue
|TradingBook|102|1,096,812|1,096,812|10,753|Db.Get
|Equity:NPV|15|440,283|765|29,352|GetValue
|Equity:NPV|15|439,517|478|29,301|GetValue/Calc
|MarketInterface:spot|15|402,755|740|26,850|GetValue
|MarketInterface:spot|15|402,014|432|26,800|GetValue/Calc
|ExternalRefData:state|15|370,076|608|24,671|GetValue
|ExternalRefData:state|15|369,467|426|24,631|GetValue/Calc
|Clock:cutoffs|870|262,795|13,465|302|GetValue
|Clock:cutoffs|16|249,548|490|15,596|GetValue/Calc
|Clock:parent|16|246,581|603|15,411|GetValue
|Clock:parent|16|245,977|485|15,373|GetValue/Calc
|RefData:state|15|215,206|347|14,347|GetValue
|RefData:state|4|214,859|157|53,714|GetValue/Calc
|Root:Amend Portfolio|11|203,634|1,876|18,512|Context
|Root:Amend Portfolio:4478373784|144|200,294|176,897|1,390|Context
|RefDataUpdateEvent|9|147,506|147,506|16,389|Db.Get
|PortfolioUpdateEvent|12|143,263|143,263|11,938|Db.Get
|_WorkItemEvent:book2|4,052|89,769|75,516|22|GetValue
|_WorkItemEvent:book1|4,052|80,932|80,932|19|GetValue
|TradingBook:clock|701|79,576|27,903|113|GetValue
|_WorkItemEvent:item|2,026|78,647|64,515|38|GetValue
|TradingPortfolio|10|74,083|74,083|7,408|Db.Get
|TradeOpenEvent:quantity|4,052|71,546|71,546|17|GetValue
|TradeOpenEvent:premium|2,026|55,716|48,581|27|GetValue
|TradingBook:clock|500|51,673|17,976|103|GetValue/Calc
|TradeOpenEvent:action|2,026|51,393|51,393|25|GetValue
|Root:Start|20|49,938|48,192|2,496|Context
|Root:End|27|47,358|45,071|1,754|Context
|Clock|5|45,747|45,747|9,149|Db.Get
|TradeOpenEvent:unitPrice|2,026|42,804|42,804|21|GetValue
|Event:amends|2,083|37,060|37,060|17|GetValue
|Equity:refdata|15|33,560|620|2,237|GetValue
|Equity:refdata|15|32,940|447|2,196|GetValue/Calc
|MarketInterface|2|30,181|30,181|15,090|Db.Get
|MarketInterface:source|15|28,788|552|1,919|GetValue
|MarketInterface:source|15|28,236|437|1,882|GetValue/Calc
|MarketDataSource|2|23,564|23,564|11,782|Db.Get
|ClockEvent|2|22,089|22,089|11,044|Db.Get
|Root:Amend MarketData|11|19,764|2,194|1,796|Context
|ClockEvent:parent|6|19,475|245|3,245|GetValue
|Root:Amend MarketData:4478373784|2|16,721|16,339|8,360|Context
|MarketDataSource:clock|23|14,325|938|622|GetValue
|Portfolio:clock|140|14,287|3,687|102|GetValue
|Equity|2|14,132|14,132|7,066|Db.Get
|MarketDataSource:clock|19|13,387|664|704|GetValue/Calc
|Portfolio:clock|44|10,600|1,321|240|GetValue/Calc
|Portfolio:books|63|8,379|1,921|133|GetValue
|RootClock|1|8,020|8,020|8,020|Db.Get
|ForwardCashflow|1|7,134|7,134|7,134|Db.Get
|Portfolio:books|33|6,938|967|210|GetValue/Calc
|Root:Activity Trading|18|5,300|3,592|294|Context
|Root:Activity Portfolio|18|4,091|2,879|227|Context
|Entity:clock|32|2,346|1,010|73|GetValue
|Root:Amend Trading|11|2,341|1,620|212|Context
|Root:Activity MarketData|11|2,333|1,619|212|Context
|Root:Start breaks|11|2,318|1,587|210|Context
|MarketInterface:sourceName|15|1,341|588|89|GetValue
|ForwardCashflow:NPV|11|1,338|495|121|GetValue
|Entity:clock|16|1,335|582|83|GetValue/Calc
|ForwardCashflow:NPV|11|843|333|76|GetValue/Calc
|MarketInterface:sourceName|15|753|440|50|GetValue/Calc
|RootClock:cutoffs|43|728|581|16|GetValue
|Equity:assetName|15|379|379|25|GetValue
|RefDataUpdateEvent:data|15|297|297|19|GetValue
|RootClock:cutoffs|1|147|80|147|GetValue/Calc
|RootClock:cosmicAll|1|38|21|38|GetValue
|CosmicAll:dbState|1|28|17|28|GetValue
|PnLExplainReport:valuable|2|20|20|10|GetValue
|PnLExplainReport:ts1|2|19|19|9|GetValue
|RootClock:cosmicAll|1|16|16|16|GetValue/Calc
|PnLExplainReport:ts2|2|15|15|7|GetValue
|CosmicAll:dbState|1|10|10|10|GetValue/Calc